In [1]:
import pandas as pd
import plotly.express as px

predictions = pd.read_parquet("output/predictions.parquet")
predictions["vote_correct"] = predictions["prediction"] == predictions["ground_truth"]

In [2]:
party_line_correct_plot_data = predictions.groupby("party")["vote_correct"].mean()
party_line_correct_plot_data["average"] = predictions["vote_correct"].mean()


df = party_line_correct_plot_data.reset_index().sort_values(by='vote_correct', ascending=False)
df.columns = ['Party', 'Correct']
df['Percent'] = (df['Correct'] * 100).round(1).astype(str) + '%'
color_map = {
    'DIE_LINKE': '#BE3075',
    'AfD': '#00A2DE',
    'Union': '#000000',
    'FDP': '#FFED00',
    'DIE_GRÜNEN': '#409A3C',
    'SPD': '#E3000F',
    'average': '#808080'
}

fig = px.bar(
    df,
    x='Party',
    y='Correct',
    color='Party',
    color_discrete_map=color_map,
    text='Percent'
)

fig.update_layout(
    xaxis_title='Party',
    yaxis_title='Prozent',
    showlegend=False,
    title='Wie oft halten sich die Parteien an das eigene Wahlprogramm?',
    bargap=0.4
)

fig.show()


In [3]:
by_category = predictions.groupby("category")["vote_correct"].mean().sort_values()

fig = px.bar(
    by_category,
    x=[cat.split("-")[0].strip() for cat in by_category.index],
    y=by_category.values,
    color=by_category.index,
    text=[f"{v:.2%}" for v in by_category.values],
    labels={"x": "Category", "y": "Vote Correctness"},
    title="Vote Correctness by Category",
)

fig.update_layout(
    xaxis_title="Category",
    yaxis_title="Vote Correctness",
    showlegend=False
)

fig.show()

In [4]:
import plotly.graph_objects as go


df = (
    predictions.groupby(["is_governing", "category"])["vote_correct"]
    .mean()
    .to_frame()
    .reset_index()
    .sort_values(by=["is_governing", "vote_correct"], ascending=[True, False])
)

votes_per_category = predictions.groupby("category")["vote_id"].nunique()
statistically_relevant = votes_per_category[votes_per_category > 10].index
df = df[df["category"].isin(statistically_relevant)]

# assume df is your DataFrame
# sort categories so the largest gaps stand out (optional)
# you could sort by abs difference:
diffs = df.pivot(index="category", columns="is_governing", values="vote_correct")
diffs["gap"] = (diffs[True] - diffs[False]).abs()
ordered_cats = diffs.sort_values("gap", ascending=False).index.tolist()
df["category"] = pd.Categorical(df["category"], categories=ordered_cats, ordered=True)

fig = go.Figure()

# add a line for each category
for cat in ordered_cats:
    sub = df[df["category"] == cat]
    fig.add_trace(
        go.Scatter(
            x=sub["vote_correct"],
            y=[cat, cat],
            mode="lines",
            line=dict(color="lightgray"),
            showlegend=False,
            hoverinfo="none",
        )
    )

# add the two sets of points
fig.add_trace(
    go.Scatter(
        x=df[df["is_governing"] == True]["vote_correct"],
        y=df[df["is_governing"] == True]["category"],
        mode="markers",
        name="Governing",
        marker=dict(symbol="circle", size=10),
    )
)
fig.add_trace(
    go.Scatter(
        x=df[df["is_governing"] == False]["vote_correct"],
        y=df[df["is_governing"] == False]["category"],
        mode="markers",
        name="Opposition",
        marker=dict(symbol="circle-open", size=10),
    )
)

fig.update_layout(
    title="Vote-correctness: Government vs. Opposition by Category",
    xaxis_title="Vote Correctness",
    yaxis_title="Category",
    margin=dict(l=250, r=50, t=50, b=50),
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
)

fig.show()


In [5]:
predictions.query("category == 'Wohnen & Stadtentwicklung - Wohnungsbau, Städtebau, Bauordnung, Städtebauförderung'")

,vote_id,type,title,drucksache_id,beschlussempfehlung,summary,summary_embedding,date,party,reasoning,prediction,ground_truth,category,is_governing,bundestag,vote_correct
120,20210507_1,Änderungsantrag,Änderungsantrag der Abgeordneten Christian Küh...,19/29409,None,Es wird über die Aufhebung von § 13b des Bauge...,"[0.01880316436290741, 0.045680250972509384, 0....",2021-05-07,AfD,{'context': '- Verbot der Einflussnahme von Pa...,Ablehnung,Ablehnung,"Wohnen & Stadtentwicklung - Wohnungsbau, Städt...",False,19,True
204,20181129_5,Änderungsantrag,Gesetzentwurf der Bundesregierung Entwurf eine...,19/4672,None,"Es wird über ein Gesetz abgestimmt, das die Re...","[0.0016128038987517357, 0.0373695082962513, 0....",2018-11-29,AfD,"{'context': '- Wirtschafts-, Steuer- und Finan...",Ablehnung,Ablehnung,"Wohnen & Stadtentwicklung - Wohnungsbau, Städt...",False,19,True
205,20181129_4,Änderungsantrag,Gesetzentwurf der Bundesregierung Entwurf eine...,19/4672,None,Im Bundestag wird über ein Gesetz zur Ergänzun...,"[0.0016812053509056568, 0.04910259321331978, 0...",2018-11-29,AfD,"{'context': '- Wirtschafts-, Steuer- und Finan...",Ablehnung,Ablehnung,"Wohnen & Stadtentwicklung - Wohnungsbau, Städt...",False,19,True
206,20181129_3,Änderungsantrag,"Änderungsantrag der Abgeordneten Lisa Paus, Ch...",19/6156,None,Es wird über einen Änderungsantrag zum Gesetz ...,"[0.0094747394323349, 0.04133496433496475, 0.02...",2018-11-29,AfD,"{'context': '- Infrastruktur, Verkehr und Wohn...",Ablehnung,Ablehnung,"Wohnen & Stadtentwicklung - Wohnungsbau, Städt...",False,19,True
334,20200514_2,Antrag,Antrag der Abgeordneten Dr. Wolfgang Strengman...,19/18939,Ablehnung,Abgestimmt wird über Maßnahmen zur schnellen u...,"[-0.040159158408641815, 0.0019875725265592337,...",2020-05-14,AfD,{'context': '- Sozialpolitik und Gesundheit: B...,Ablehnung,Ablehnung,"Wohnen & Stadtentwicklung - Wohnungsbau, Städt...",False,19,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3412,20160929_1,Antrag,"Antrag der Abgeordneten Caren Lay, Herbert Beh...",18/8863,Ablehnung,Abgestimmt wird über die Einbringung eines zwe...,"[0.01597343385219574, 0.05057711899280548, 0.0...",2016-09-29,Union,"{'context': '- **Lebenswerte Heimat, Umwelt un...",Annahme,Ablehnung,"Wohnen & Stadtentwicklung - Wohnungsbau, Städt...",True,18,False
3443,20150327_7,Antrag,Antrag der Abgeordneten Christian Kühn (Tübing...,18/3044,Ablehnung,Abgestimmt wird über die Einführung einer nach...,"[0.017921751365065575, 0.0666632354259491, 0.0...",2015-03-27,Union,"{'context': '- **Lebenswerte Heimat, Umwelt un...",Annahme,Ablehnung,"Wohnen & Stadtentwicklung - Wohnungsbau, Städt...",True,18,False
3472,20130606,Antrag,"Antrag der Abgeordneten Michael Groß, Sören Ba...",17/12485,Ablehnung,"Der Bundestag stimmt über Maßnahmen ab, die be...","[-7.3927662924688775e-06, -0.00145466974936425...",2013-06-06,Union,{'context': '- Stärkung der Investitionsfähigk...,Annahme,Ablehnung,"Wohnen & Stadtentwicklung - Wohnungsbau, Städt...",True,17,False
3473,20130606,Antrag,"Antrag der Abgeordneten Heidrun Bluhm, Halina ...",17/12481,Ablehnung,Der Bundestag stimmt über Maßnahmen zur Bekämp...,"[0.015009190887212753, 0.03039916418492794, 0....",2013-06-06,Union,"{'context': '- Bekämpfung von Protektionismus,...",Annahme,Ablehnung,"Wohnen & Stadtentwicklung - Wohnungsbau, Städt...",True,17,False


In [6]:
row = predictions.iloc[3478]
row

vote_id                                                         20130315
type                                                              Antrag
title                  Antrag der Abgeordneten Sylvia Kotting-Uhl, Bä...
drucksache_id                                                   17/11206
beschlussempfehlung                                            Ablehnung
summary                Der Bundestag soll über die Aufnahme bilateral...
summary_embedding      [0.0034483608324080706, 0.0238827932626009, 0....
date                                                 2013-03-15 00:00:00
party                                                              Union
reasoning              {'context': '- Anpassung des Jugendstrafrechts...
prediction                                                       Annahme
ground_truth                                                   Ablehnung
category               Umwelt, Klima & Naturschutz - Umweltschutz, Kl...
is_governing                                       

In [7]:
row.summary

'Der Bundestag soll über die Aufnahme bilateraler Verhandlungen mit Frankreich abstimmen, um die unverzügliche Stilllegung der besonders gefährlichen und alten grenznahen Atomkraftwerke Fessenheim und Cattenom zu erreichen. Es wird gefordert, die Sicherheitsberichte und EU-Stresstestergebnisse systematisch auszuwerten und die Ergebnisse in die Zusammenarbeit mit Frankreich für Nachrüstungen oder weitere Stilllegungen einzubringen. Ziel ist eine verbesserte Kooperation im Bereich Nuklearsicherheit, ein systematischer Austausch wesentlicher Sicherheitsunterlagen sowie eine deutlich stärkere Berücksichtigung der Risiken grenznaher ausländischer AKWs zum Schutz der Bevölkerung in Deutschland. Außerdem soll die Bundesregierung die Öffentlichkeit besser über Sicherheitsrisiken und ihre Aktivitäten informieren.'

In [8]:
from scipy.stats import ttest_rel, wilcoxon

opp = [1.000000, 0.666667, 0.666667, 0.333333, 1.000000, 1.000000, 0.500000, 1.000000, 1.000000, 1.000000, 1.000000, 1.000000, 0.750000, 0.750000]
reg = [0.000000, 0.500000, 0.500000, 0.500000, 0.500000, 0.000000, 0.000000, 0.000000, 0.000000, 0.500000, 0.500000, 0.500000, 0.500000, 0.500000]

t_stat, t_p = ttest_rel(opp, reg)
w_stat, w_p = wilcoxon(opp, reg)

print(f"Paired t-test: t = {t_stat:.4f}, p = {t_p:.6f}")
print(f"Wilcoxon signed-rank test: W = {w_stat}, p = {w_p:.6f}")

Paired t-test: t = 5.1622, p = 0.000183
Wilcoxon signed-rank test: W = 2.0, p = 0.001384


In [9]:
!pip install scipy